In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
batch_size = 32
block_size = 128
epochs = 10000
eval_interval = 500
learning_rate = 0.0003
eval_iters = 20
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [38]:
torch.cuda.empty_cache()


In [39]:
with open(r"C:\Users\micah\Desktop\tiny-shakespeare.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [40]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

#text = text.replace('\n', ' ')
#text = text.replace(':', ' ')
#words = text.split(' ')

65

In [41]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
#num_words = len(words)

In [42]:
data = torch.tensor(encode(text))
n = int(.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [43]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+1 + block_size] for i in ix])
    #x = x.to(device)
    #y = y.to(device)
    return x, y


In [44]:
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X = X.to(device)
            Y = Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [45]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * C ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


In [46]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=2)
        out = self.dropout(self.proj(out))
        return out

In [47]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [48]:
class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd // n_heads
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        #print(x.shape)
        #print(self.sa(self.ln1(x)).shape)
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [49]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (Batch,T = Block,C = embedding)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))# (T,C)
        #pos_emb = self.position_embedding_table(torch.arange(T).to(device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [50]:

model = Transformer()
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


10.739777 M parameters


In [51]:
model.to(device)

Transformer(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(128, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True

In [56]:

for iter in range(epochs):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model


step 0: train loss 1.0929, val loss 1.5069
step 49: train loss 1.0917, val loss 1.5060
step 500: train loss 1.0778, val loss 1.4837
step 1000: train loss 1.0646, val loss 1.5001
step 1500: train loss 1.0474, val loss 1.5239
step 2000: train loss 1.0353, val loss 1.4950
step 2500: train loss 1.0213, val loss 1.5165
step 3000: train loss 1.0190, val loss 1.5099
step 3500: train loss 0.9983, val loss 1.5558
step 4000: train loss 0.9820, val loss 1.5412
step 4500: train loss 0.9719, val loss 1.5594
step 5000: train loss 0.9624, val loss 1.5676
step 5500: train loss 0.9549, val loss 1.5554
step 6000: train loss 0.9214, val loss 1.5763
step 6500: train loss 0.9226, val loss 1.5483
step 7000: train loss 0.9115, val loss 1.5611
step 7500: train loss 0.9027, val loss 1.6094
step 8000: train loss 0.8765, val loss 1.5866
step 8500: train loss 0.8678, val loss 1.5725
step 9000: train loss 0.8583, val loss 1.6398
step 9500: train loss 0.8444, val loss 1.6252


In [57]:
context = torch.zeros((1, block_size), dtype=torch.long).to(device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

































































































































ISAMPSOPELE:
MAMPSARy mind; I better do come to win them. I pray you, lords.

CAMILLO:
If you'll play for't.

Shepherd:
Since of thine, one speak?

CLARENCE:
Neither to Barne.

GLOUCESTER:
I wish my lord, and learned place out.

GLOUCESTER:
Widow you presently and your fair affairs?

LADY ANNE:
And, well, do I send my lady son
Hast to deserved my grew!

GLOUCESTER:
Cousin, of Buckingham, I will be pardon'd,
When he sin from Edward's shroud: adie all effect,
Brags of our readiness he shall stand 
